In [6]:
import json
from functools import lru_cache
from tqdm import tqdm

from domain import *
from helper import *

In [3]:
splits = ['train', 'val', 'test']

split2dataset = {}
for split in splits:
    split2dataset[split] = json.load(open('expr_%s.json'%split))

In [8]:
@lru_cache
def compute_max_dep(heads):
    return max([0] + [abs(i-h) for i, h in enumerate(heads) if h != -1])

@lru_cache
def compute_tree_depth(head):
    @lru_cache
    def depth(i):
        """The depth of node i."""
        if head[i] == -1:
            return 1
        return depth(head[i]) + 1

    return max(depth(i) for i in range(len(head)))

lps = '('
rps = ')'
def compute_ps_depth(expr):
    depth = 0
    max_depth = 0
    for x in expr:
        if x == lps:
            c = 1
        elif x == rps:
            c = -1
        else:
            c = 0
        depth += c
        if depth > max_depth:
            max_depth = depth
    return max_depth

def compute_n_op(expr):
    return len([1 for x in expr if x in OPERATORS])

In [13]:
for split, dataset in split2dataset.items():
    print(split)
    for sample in tqdm(dataset):
        sample['tree_depth'] = compute_tree_depth(tuple(sample['head']))
        sample['ps_depth'] = compute_ps_depth(sample['expr'])
        sample['n_op'] = compute_n_op(sample['expr'])
        sample['max_dep'] = compute_max_dep(tuple(sample['head']))
        sample['length'] = len(sample['expr'])

json.dump(split2dataset, open('dataset_with_properties.json', 'w'))

  5%|█▌                              | 49986/998000 [00:00<00:01, 499833.07it/s]

train


  0%|                                                 | 0/46620 [00:00<?, ?it/s]

val
test


100%|██████████████████████████████████| 46620/46620 [00:01<00:00, 45634.62it/s]
